In [3]:
# Library import
import pandas as pd
import numpy as np
import os

In [1]:
# For drive input
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
# Data reading

#data_path = "/content/drive/MyDrive/DS Shared/FCS" # for drive input
#data_path = ""

dogs = pd.read_csv(os.path.join(data_path, "datasets/dogs.csv"))
dogs_travel = pd.read_csv(os.path.join(data_path, "datasets/dogTravel.csv"), index_col=0)
populations = pd.read_csv(os.path.join(data_path, "datasets/NST-EST2021-POP.csv"), header=None, names=['State', 'pop'])

In [18]:
# Display df header and variables info
display(dogs.head())
dogs.info()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58180 entries, 0 to 58179
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               58180 non-null  int64  
 1   org_id           58180 non-null  object 
 2   url              58180 non-null  object 
 3   type.x           58180 non-null  object 
 4   species          58180 non-null  object 
 5   breed_primary    58180 non-null  object 
 6   breed_secondary  20821 non-null  object 
 7   breed_mixed      58180 non-null  bool   
 8   breed_unknown    58180 non-null  bool   
 9   color_primary    26134 non-null  object 
 10  color_secondary  12059 non-null  object 
 11  color_tertiary   1217 non-null   object 
 12  age              58180 non-null  object 
 13  sex              58180 non-null  object 
 14  size             58180 non-null  object 
 15  coat             27185 non-null  object 
 16  fixed            58180 non-null  bool   
 17  house_traine

## Data Cleansing
### Dogs

In [19]:
# 'status' attribute had no values for 'not adoptable' dogs so the ohther attributes shifted to the left

# find not adoptable dogs
misread = dogs['status'] != 'adoptable'
# shift their attributes to the right
temp = dogs.shift(1, axis=1).loc[misread]
# adjust the dataset
dogs.loc[misread, dogs.columns[26:33]] = temp.iloc[:, 26:33]
dogs.loc[misread, 'status'] = 'not adoptable'

In [20]:
# Change the date format to datetime
dogs.posted = pd.to_datetime(dogs.posted.apply(lambda x: x.replace("T", " ")))
dogs.accessed = pd.to_datetime(dogs.accessed, utc=True)

### DogTravel

In [21]:
display(dogs_travel.head(10))
dogs_travel.info()

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
5,43082511,Manchester,CT,Brooke has an unusual past. She was rescued f...,Afghanistan,NaN,NaN,NaN
6,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
7,45287347,Wooster,OH,"Tate is an adorable 2 year old, 22 pound Cocka...",Akron,Ohio,NaN,NaN
8,45987719,Locust Fork,AL,Meet Trixie... she is a female 2yr. Old Chihua...,Alabama,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6194 entries, 0 to 6193
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             6194 non-null   int64 
 1   contact_city   6194 non-null   object
 2   contact_state  6194 non-null   object
 3   description    6194 non-null   object
 4   found          6194 non-null   object
 5   manual         2147 non-null   object
 6   remove         1738 non-null   object
 7   still_there    319 non-null    object
dtypes: int64(1), object(7)
memory usage: 435.5+ KB


### Populations

In [22]:
# Convert population sizes from strings to numbers
def str_to_number(x):
    return x.replace('.', '')

populations['pop'] = populations['pop'].apply(str_to_number).astype(np.float32)

## Project

In [25]:
# 1. Extract all dogs with status that is not adoptable

dogs[dogs["status"] == "not adoptable"][["id", "status"]]

,id,status
644,41330726,not adoptable
5549,38169117,not adoptable
10888,45833989,not adoptable
11983,45515547,not adoptable
12495,45294115,not adoptable
12600,45229004,not adoptable
12613,45227052,not adoptable
17619,45569380,not adoptable
18611,44694387,not adoptable
19747,36978896,not adoptable


In [26]:
# 2. For each (primary) breed, determine the number of dogs

dogs.groupby('breed_primary').count()['id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

In [75]:
# 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

# We noticed that non Mixed breed dogs had nan value for attribute 'Secondary Breed'
# We decided to compute the ratio, for every value of 'Primary Breed', between the number of not nan and nan
def mixb_ratio(x):
    if sum(x.isnull()) != 0:
        return sum(1 - pd.isna(x)) / sum(pd.isna(x))
    else:
        return np.inf

dogs.groupby('breed_primary')['breed_secondary'].apply(mixb_ratio)

breed_primary
Affenpinscher                        0.133333
Afghan Hound                         0.333333
Airedale Terrier                     0.900000
Akbash                               0.000000
Akita                                0.403101
                                       ...   
Wirehaired Pointing Griffon          0.000000
Wirehaired Terrier                   0.428571
Xoloitzcuintli / Mexican Hairless    0.571429
Yellow Labrador Retriever            0.645833
Yorkshire Terrier                    0.558442
Name: breed_secondary, Length: 216, dtype: float64

In [76]:
# 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

# Find the latest timestamp for each primary breed
min_date = dogs.iloc[(dogs.groupby("breed_primary")["posted"].idxmax())][["breed_primary", "posted"]]
# Find the earliest timestamp for each primary breed
max_date = dogs.iloc[(dogs.groupby("breed_primary")["posted"].idxmin())][["breed_primary", "posted"]]

# Merge the two and change the suffixes
pd.merge(min_date, max_date, on='breed_primary', suffixes=['_earliest', '_latest'])

,breed_primary,posted_earliest,posted_latest
0,Affenpinscher,2019-09-14 10:10:51+00:00,2012-03-08 10:27:33+00:00
1,Afghan Hound,2019-07-27 00:38:48+00:00,2017-06-29 23:28:51+00:00
2,Airedale Terrier,2019-09-19 18:40:39+00:00,2014-06-13 12:59:36+00:00
3,Akbash,2019-08-23 17:11:04+00:00,2019-07-21 00:35:59+00:00
4,Akita,2019-09-20 15:19:57+00:00,2012-03-03 09:31:08+00:00
...,...,...,...
211,Wirehaired Pointing Griffon,2016-06-29 20:03:55+00:00,2016-06-29 20:03:55+00:00
212,Wirehaired Terrier,2019-09-19 22:52:45+00:00,2012-11-27 14:07:54+00:00
213,Xoloitzcuintli / Mexican Hairless,2019-09-08 11:15:54+00:00,2007-02-01 00:00:00+00:00
214,Yellow Labrador Retriever,2019-09-20 06:30:27+00:00,2010-05-31 00:00:00+00:00


In [30]:
# 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

# Find male dogs
male_dogs = dogs[dogs['sex']== 'Male']
# Find female dogs
female_dogs = dogs[dogs['sex']== 'Female']

# Count male dogs for each state
male_count = male_dogs.groupby(['contact_state']).count()['id']
# Count female dogs for each state
female_count = female_dogs.groupby(['contact_state']).count()['id']

# Print the state with the larget sex imbalance
print(f"The state with the largesr sex imbalance is {(abs(male_count-female_count)).idxmax()}")

The state with the largesr sex imbalance is OH


In [78]:
# 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

dogs.groupby(['age', 'size'])[['stay_duration', 'stay_cost']].mean()

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

In [31]:
# 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

# Join travels with dogs info
joined = pd.merge(dogs, dogs_travel, on="id")

# Count the amount of travels for each dog
n_travels = joined.groupby("id", as_index= False).size()

# Attach the n travels to the dogs table
joined2 = pd.merge(n_travels, dogs, on = "id")

# Print dogs with at least 3 travels
joined2[joined2["size_x"] > 2][["id", "size_x", "breed_primary", "breed_secondary"]]

,id,size_x,breed_primary,breed_secondary
5,16657005,4,Pit Bull Terrier,NaN
9,20905974,5,Chow Chow,English Springer Spaniel
17,24894870,4,Hound,Terrier
18,24894894,4,Hound,Terrier
55,33218331,7,Alaskan Malamute,Siberian Husky
...,...,...,...,...
4110,46042569,3,Labrador Retriever,NaN
4111,46042587,3,Labrador Retriever,NaN
4112,46042618,3,Labrador Retriever,NaN
4113,46043099,3,Labrador Retriever,NaN


In [32]:
# 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

dogs_travel.where(~dogs_travel.manual.apply(pd.isna), dogs_travel.found, axis=0)

,id,contact_city,contact_state,description,found,manual,remove,still_there
index,,,,,,,,
0,Arkansas,Arkansas,Arkansas,Arkansas,Arkansas,Arkansas,Arkansas,Arkansas
1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,Adaptil,Adaptil,Adaptil,Adaptil,Adaptil,Adaptil,Adaptil,Adaptil
4,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan
...,...,...,...,...,...,...,...,...
6189,WV,WV,WV,WV,WV,WV,WV,WV
6190,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming,Wyoming
6191,Yazmin,Yazmin,Yazmin,Yazmin,Yazmin,Yazmin,Yazmin,Yazmin


In [33]:
# 9. For each state, compute the ratio between the number of travels and the population.

In [34]:
# Definition of the lookup table to convert state names to postal codes
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
populations['state_code'] = populations.State.replace(us_state_to_abbrev)

In [35]:
# Count the travels for each state
#moved = pd.isna(dogs_travel.still_there)
counts = dogs_travel.groupby('contact_state')['id'].count() #.loc[moved, :]

In [40]:
# Join populations with counts
temp = populations.set_index('state_code').join(counts).fillna(0) # States with no travels are filled with 0
ratios = temp['id'] / temp['pop']

In [41]:
# Sorting ratios from the greatest to the lowest
ratios.sort_values(ascending=False)

state_code
ME    2.649816e-04
DC    1.624259e-04
VA    1.187526e-04
RI    1.184641e-04
WA    8.228123e-05
VT    7.619616e-05
MD    6.135442e-05
NJ    5.942516e-05
DE    5.757878e-05
NM    3.872451e-05
MN    3.329540e-05
NH    3.194125e-05
CT    2.495879e-05
PA    2.430264e-05
NY    2.425593e-05
UT    2.017352e-05
NC    1.791293e-05
CO    1.783947e-05
TN    1.620642e-05
MA    1.522066e-05
WV    1.505255e-05
OH    1.500070e-05
AL    1.492751e-05
WI    1.408279e-05
GA    1.017559e-05
AZ    9.788154e-06
IN    9.579210e-06
OK    7.829562e-06
OR    7.552057e-06
IA    6.895754e-06
KY    6.879966e-06
MO    6.823817e-06
MI    6.648586e-06
NV    6.442025e-06
FL    6.175078e-06
SC    4.102825e-06
AR    3.320578e-06
IL    2.887803e-06
MS    2.026152e-06
LA    1.073478e-06
KS    1.021144e-06
CA    7.081755e-07
TX    6.862122e-08
ID    0.000000e+00
HI    0.000000e+00
SD    0.000000e+00
NE    0.000000e+00
AK    0.000000e+00
MT    0.000000e+00
ND    0.000000e+00
WY    0.000000e+00
dtype: float64

In [43]:
# 10. For each dog, compute the number of days from the "posted" day to the day of last access.

# Compute the differene from accessed and posted timestamps in days
dogs["access_posted_diff"] = (dogs["accessed"].dt.date)-(dogs["posted"].dt.date) 

# Print results
dogs[["id", "access_posted_diff"]]

,id,access_posted_diff
0,46042150,0 days
1,46042002,0 days
2,46040898,0 days
3,46039877,0 days
4,46039306,0 days
...,...,...
58175,44605893,140 days
58176,44457061,160 days
58177,42865848,358 days
58178,42734734,373 days


In [109]:
# 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.
#from datetime import datetime

# Changing the difference computed in the previous task into number of weeks
dogs["access_posted_weeks"] = dogs.access_posted_diff.apply(lambda x: int(pd.Timedelta(x).days / 7))

# Partition the df
dogs.set_index(['access_posted_weeks', 'id'])

org_id  \
access_posted_weeks id                
0                   46042150  NV163   
                    46042002  NV163   
                    46040898   NV99   
                    46039877  NV202   
                    46039306  NV184   
...                             ...   
20                  44605893   WY20   
22                  44457061   WY24   
51                  42865848   WY20   
53                  42734734   WY24   
54                  42663515   WY24   

                                                                            url  \
access_posted_weeks id                                                            
0                   46042150  https://www.petfinder.com/dog/harley-46042150/...   
                    46042002  https://www.petfinder.com/dog/biggie-46042002/...   
                    46040898  https://www.petfinder.com/dog/ziggy-46040898/n...   
                    46039877  https://www.petfinder.com/dog/gypsy-46039877/n...   
                    46039306  https://www.petfinder.com/dog/theo-46039306/nv...   
...                                                                         ...   
20                  44605893  https://www.petfinder.com/dog/tren-44605893/wy...   
22                  44457061  https://www.petfinder.com/dog/harley-44457061/...   
51                  42865848  https://www.petfinder.com/dog/echo-42865848/wy...   
53                  42734734  https://www.petfinder.com/dog/simon-42734734/w...   
54                  42663515  https://www.petfinder.com/dog/romeo-42663515/w...   

                             type.x species                   breed_primary  \
access_posted_weeks id                                                        
0                   46042150    Dog     Dog  American Staffordshire Terrier   
                    46042002    Dog     Dog                Pit Bull Terrier   
                    46040898    Dog     Dog                        Shepherd   
                    46039877    Dog     Dog             German Shepherd Dog   
                    46039306    Dog     Dog                       Dachshund   
...                             ...     ...                             ...   
20                  44605893    Dog     Dog                   Border Collie   
22                  44457061    Dog     Dog             Australian Shepherd   
51                  42865848    Dog     Dog                   Border Collie   
53                  42734734    Dog     Dog                           Boxer   
54                  42663515    Dog     Dog              Labrador Retriever   

                                                  breed_secondary  \
access_posted_weeks id                                              
0                   46042150                          Mixed Breed   
                    46042002                          Mixed Breed   
                    46040898                                  NaN   
                    46039877                                  NaN   
                    46039306                                  NaN   
...                                                           ...   
20                  44605893                                  NaN   
22                  44457061  Australian Cattle Dog / Blue Heeler   
51                  42865848                                  NaN   
53                  42734734                          Mixed Breed   
54                  42663515                     Pit Bull Terrier   

                              breed_mixed  breed_unknown  \
access_posted_weeks id                                     
0                   46042150         True          False   
                    46042002         True          False   
                    46040898        False          False   
                    46039877        False          False   
                    46039306        False          False   
...                                   ...            ...   
20                  44605893       

In [ ]:
# 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.
from tqdm import tqdm

# Function to compute the % of words shared by the two descriptions
def perc_of_equal_words(x, y):
    s = 0
    if pd.isnull(x) or pd.isnull(y):
        return int(x == y)
    else:
        for wx in str.split(x):
            for wy in str.split(y):
                if wx == wy:
                    s += 1
        return s / max(len(x), len(y)) # % is computed by dividing the number of shared word by the max lenght of the two descriptions

In [115]:
# Define the dataset to include possible duplicates
df = {
    'id1': [],
    'id2': [],
    'perc': []
}

nd_dogs = dogs[pd.isna(dogs.description)] # dogs with null description
nnd_dogs = dogs[~pd.isna(dogs.description)] # dogs with non-null description

In [112]:
# For null description dogs, duplicates share the same sex, primary, and secondary breed
nd_dogs[nd_dogs.duplicated(subset=['breed_primary', 'breed_secondary', 'sex'])]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,access_posted_diff,access_posted_weeks
17,46038067,NV26,https://www.petfinder.com/dog/marie-46038067/n...,Dog,Dog,Pit Bull Terrier,NaN,True,False,NaN,...,89101,US,89009,2019-09-20 00:00:00+00:00,Dog,NaN,136,231.52,0 days,0
20,46038062,NV26,https://www.petfinder.com/dog/duncan-46038062/...,Dog,Dog,Pit Bull Terrier,NaN,True,False,NaN,...,89101,US,89009,2019-09-20 00:00:00+00:00,Dog,NaN,80,91.22,0 days,0
21,46038063,NV26,https://www.petfinder.com/dog/o-and-number-39c...,Dog,Dog,Pit Bull Terrier,NaN,True,False,NaN,...,89101,US,89009,2019-09-20 00:00:00+00:00,Dog,NaN,88,353.15,0 days,0
32,46036459,NV26,https://www.petfinder.com/dog/crockett-4603645...,Dog,Dog,Boxer,NaN,True,False,NaN,...,89101,US,89009,2019-09-20 00:00:00+00:00,Dog,NaN,93,400.86,1 days,0
35,46035353,NV26,https://www.petfinder.com/dog/tonka-46035353/n...,Dog,Dog,Dachshund,Mixed Breed,True,False,NaN,...,89101,US,89009,2019-09-20 00:00:00+00:00,Dog,NaN,89,135.90,1 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58173,44629286,WY04,https://www.petfinder.com/dog/suzi-44629286/wy...,Dog,Dog,Labrador Retriever,Hound,True,False,Brown / Chocolate,...,82401,US,WY,2019-09-20 00:00:00+00:00,Dog,NaN,61,167.10,137 days,19
58174,44629272,WY04,https://www.petfinder.com/dog/cheyenne-4462927...,Dog,Dog,Shepherd,Australian Cattle Dog / Blue Heeler,True,False,Brown / Chocolate,...,82401,US,WY,2019-09-20 00:00:00+00:00,Dog,NaN,97,242.85,137 days,19
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,82501,US,WY,2019-09-20 00:00:00+00:00,Dog,NaN,65,245.90,160 days,22
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,82501,US,WY,2019-09-20 00:00:00+00:00,Dog,NaN,58,61.05,373 days,53


In [ ]:
# For not-null description dogs we followed another approach

# Find all possible combinations of sex, primary, and secondary breed 
all_combs = nnd_dogs[['breed_primary', 'breed_secondary', 'sex']].drop_duplicates()

checked_combs = []
dups = []

for i in tqdm(range(len(all_combs))): # iterate over all combinations
    comb = all_combs.iloc[i]

    # Create a subset with dogs that share the same combination
    subset = nnd_dogs[
        (nnd_dogs.breed_primary == comb.breed_primary) &
        (nnd_dogs.breed_secondary == comb.breed_secondary) &
        (nnd_dogs.sex == comb.sex)
    ]

    # Search for duplicates in the subset
    for j in range(len(subset)):
        for k in range(len(subset)):
            if j != k:
                dog1 = subset.iloc[j]
                dog2 = subset.iloc[k]
                if perc_of_equal_words(dog1.description, dog2.description) >= 0.9: # add dogs to duplicates if they share at least 90% of the words
                    dups.append([dog1.id, dog2.id])

100%|██████████| 4282/4282 [19:47<00:00,  3.61it/s]  


In [ ]:
# Total time: ~17'13" (AIR M1)

### Another way to do 12

In [44]:
# Faster approach
dogs2 = dogs.set_index('id') # set dog index to id
nd_dogs = dogs2[pd.isna(dogs2.description)] # null description dogs
nnd_dogs = dogs2[~pd.isna(dogs2.description)] # not-null description dogs

In [45]:
import nltk
nltk.download('punkt') # download of punctiation
nltk.download('stopwords') # download of stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

stop_words = set(stopwords.words('english')) # select english stopwords
porter = PorterStemmer() # stemmer initialization
tokenizer = RegexpTokenizer(r'\w+') # tokenizer initialization

In [47]:
def text_processing(x, tokenizer, stop_words):
    word_tokens = tokenizer.tokenize(x) # tokenize word
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] # remove stopwords

    return filtered_sentence

In [49]:
# Pre-processing all descriptions (~11')
new_descs = {}
for i in tqdm(nnd_dogs.index):
    desc = nnd_dogs.loc[i].description
    new_descs[nnd_dogs.loc[i].name] = text_processing(desc, tokenizer, stop_words)

100%|██████████| 49475/49475 [00:30<00:00, 1638.48it/s]


In [50]:
# Function to compute the % of token shared
def perc_of_equal_words(x, y):
    s = 0
    for wx in x:
        for wy in y:
            if wx == wy:
                s += 1
    return s / max(len(x), len(y))

In [52]:
# Same procedure as before...
all_combs = nnd_dogs[['breed_primary', 'breed_secondary', 'sex']].drop_duplicates()
checked_combs = []
dups = []

for i in tqdm(range(len(all_combs))):
    comb = all_combs.iloc[i]

    subset = nnd_dogs[
        (nnd_dogs.breed_primary == comb.breed_primary) &
        (nnd_dogs.breed_secondary == comb.breed_secondary) &
        (nnd_dogs.sex == comb.sex)
    ]

    for j in subset.index:
        for k in subset.index:
            if j != k:
                if (j in new_descs.keys()) & (k in new_descs.keys()):
                    if perc_of_equal_words(new_descs[j], new_descs[k]) >= 0.9:
                        dups.append([j, k])

100%|██████████| 4282/4282 [04:58<00:00, 14.35it/s]


In [ ]:
# Total time: ~11" + ~2'36" = ~2':47" (AIR M1) --> -83.8% time!!!